First, let's create a small DataFrame of the land areas of the boroughs of New York City:

In [ ]:
#!conda install 

import pandas as pd
import xlsxwriter

# Create a DataFrame of land sizes of NYC boroughs
data = {'borough':['The Bronx', 'Brooklyn', 'Manhattan', 'Queens', 'Staten Island'],'land_area':[42.10,70.82,22.83,108.53,58.37]}

df = pd.DataFrame(data)

# Doesn't this look familiar?
df


In [1]:
import seaborn as sns
sns.get_dataset_names()

C:\Users\GeorgeM\anaconda3\lib\site-packages\seaborn\utils.py:384: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 384 of the file C:\Users\GeorgeM\anaconda3\lib\site-packages\seaborn\utils.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  gh_list = BeautifulSoup(http)


['anagrams',
 'anscombe',
 'attention',
 'brain_networks',
 'car_crashes',
 'diamonds',
 'dots',
 'exercise',
 'flights',
 'fmri',
 'gammas',
 'geyser',
 'iris',
 'mpg',
 'penguins',
 'planets',
 'tips',
 'titanic']

Let's say we wanted to load this DataFrame into a workbook and format the output with `xlsxwriter`.

Unfortunately, `pandas` DataFrames take a couple of extra steps to use with `xlsxwriter`. Here are our steps:

1. Set the `pandas` Excel-writing engine to `xlsxwriter` with `ExcelWriter()`.  
2. Convert the DataFrame into an `xlsxwriter` object with `to_excel()`.  
3. Create workbook and worksheet objects for the resulting output with `writer.book` and `writer.sheets`.

Let's take a look: 

In [ ]:
# Set Pandas engine to xlsxwriter
writer = pd.ExcelWriter('nycland.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df.to_excel(writer, sheet_name='Sheet1')

# Get the xlsxwriter objects from the DataFrame writer object.
workbook  = writer.book
worksheet = writer.sheets['Sheet1']

Were you to close your workbook now, you would see something like the below.

![DataFrame index visible in Excel export](images/nyc-land-index.png)


-  By default, our DataFrame will be written starting in `A1` of the worksheet. To write it elsewhere, check out this [`pandas` documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_excel.html).  


-  The numbers in column `A` represent the ***index*** of the DataFrame. Indexes are great for hleping us access and manipulate data in `pandas`, but aren't so helpful in our finished Excel export. 


We can hide the index from displaying by including `Index = False` in our `to_excel()` method.

Let's try this again:

In [ ]:
# Set Pandas engine to xlsxwriter
writer = pd.ExcelWriter('nycland.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
### index = 'False' ###
df.to_excel(writer, sheet_name='Sheet1', index=False)

# Get the xsxwriter objects from the dataframe writer object.
workbook  = writer.book
worksheet = writer.sheets['Sheet1']

workbook.close()

## Customizing `pandas` output with `xlsxwriter`

You may remember that we were writing `pandas` DataFrames to Excel back at the beginning of this course. Why do it this newfangled way now?

The benefit of sending our DataFrame to `xlsxwriter` is that we can now add any formatting and analysis to the workbook in ways that would be difficult or impossible in `pandas`. 

You've already learned several useful methods for customizing workbooks from Python. Let's learn one more. 

## Conditional formatting

Conditional formatting is a great way to add visual elements to data analysis.  

The usual disclaimer applies: You'll just get a taste of the possibilities here. For more information on conditional formatting in `xlsxwriter`, check out Chapter 22 of the [`xlsxwriter` guide](https://github.com/jmcnamara/XlsxWriter/blob/master/docs/XlsxWriter.pdf). 

In [ ]:
import pandas as pd 
import xlsxwriter

# pandas can read in data from most any source
cities = pd.read_csv("https://raw.githubusercontent.com/summerofgeorge/training-assets/master/largest-us-cities.csv")

# Convert our population field to be numeric
cities['population'] = pd.to_numeric(cities['population'], errors='coerce')

# Print the first few rows
cities.head()

## Creating workbook calculations from `pandas`

At the beginning of the class we talked about using Python and Excel *together* to make awesome data products. 

Let's discuss a bit further about how to draw the "division of labor" between these tools:

*If it has to do with data analysis, manipulation or visualization, use Python.*

*If it has to do with workbook formatting, presentation and end-user interaction, use Excel.*

We would like to add a column to our table expressing population density. We could do this in either Excel or Python, but generally, unless the end-user absolutely has to have the formulas written out in Excel, any calculations we perform should be conducted in Python. 

Fortunately, `pandas` makes it quite easy to create calculated columns. 

In [ ]:
# Create rounded population density column in our cities table

cities['density'] = cities['population']/cities['land_area']

# Check out our new column
cities.head()

Remember that the way our data looks here in `pandas` is pretty much the way it will look in Excel! So let's clean up our cell formatting as we write to the workbook.

We've been writing our custom cell formats in as we wrote data. But with `pandas`, we will be writing all our data in at once, and *then* going back to format it.

We can apply cell formatting to columns without writing to them with the `set_column()` method. This is what we used to increase column widths.

In [ ]:
# Set Pandas engine to xlsxwriter
writer = pd.ExcelWriter('us-cities.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
cities.to_excel(writer, sheet_name='Sheet1',index=False)

# Get the xlsxwriter objects from the DataFrame writer object.
workbook  = writer.book
worksheet = writer.sheets['Sheet1']

# Set up cell formatting 
thousands_format = workbook.add_format({'num_format':'#,##0'})
pct_format = workbook.add_format({'num_format':'0.00%'})

# Format our columns -- we will also set the column width here
worksheet.set_column('A:A',12)
worksheet.set_column('B:B', 10, thousands_format)
worksheet.set_column('C:C', 10, pct_format)
worksheet.set_column('D:E', 10, thousands_format)

### The basics of conditional formatting.

To add conditional formatting to our worksheet, we will apply the `conditional_format()` method to our selected cells. 

#### Greater than/less than

We would like to format all of our density values greater than 500 miles as <font color='green'>green</font>.

To do so, we will pass a dictionary of our conditional formatting settings and apply that to the range `D2:D11`.

As we've done before, we will set a workbook cell format and then include that as our option for cell formatting. 

In [ ]:
# Set green fill with dark green text format.
# bg_color will set the cell fill by hex code
green_format = workbook.add_format({'bg_color': '#C6EFCE',
'font_color': '#006100'})


worksheet.conditional_format('D2:D11', {'type': 'cell',
'criteria': 'greater than',
'value': 500,
'format': green_format})

#### Top *n*/Bottom *n*

We can also find the top or bottom *n* values in a range.

Let's find the three most and least densely-populated cities, setting them with red and green formats, respectively. 

In [ ]:
# Set top three values to red
worksheet.conditional_format('E2:E11',{'type':'top',
'value':3,
'format':green_format})

# Now we want a red format
red_format = workbook.add_format({'bg_color': '#FFC7CE',
'font_color': '#9C0006'})

# Apply red format to bottom 3
worksheet.conditional_format('E2:E11',{'type':'bottom',
'value':3,
'format':red_format})

#### Data bar

We will now add data bars to our population column. Fortunately this time, the formatting is already made for us 🎉.

There are ways to customize the data bar settings -- check the documentation for more.

In [ ]:
# Format range with data bars
worksheet.conditional_format('A2:A11', {'type': 'data_bar'})

#### Color scales

Similarly, we can apply a pre-set format color scale. Let's try it on the population change column. We will apply a three-color scale:

In [ ]:
# Format range with color scale
worksheet.conditional_format('C2:A11', {'type': '3_color_scale'})

# That's a wrap!
workbook.close()

# Drill

Complete the code scaffolding to format the following:

- Format the cells of boroughs with a land area greater than 50 square miles with a green format.
- Add a two-grade color scale to the population column.  



In [ ]:
import pandas as pd
import xlsxwriter

# Create a DataFrame of land sizes of NYC boroughs
data = {'borough':['The Bronx', 'Brooklyn', 'Manhattan', 'Queens', 'Staten Island'],'land_area':[42.10,70.82,22.83,108.53,58.37],'population':[1418207,2559903,1628706,2253858,476143]}

nyc = pd.DataFrame(data)

# Set Pandas engine to xlsxwriter
writer = ___('nyc.xlsx', engine=___)

# Convert the dataframe to an XlsxWriter Excel object.
nyc.to_excel(___, sheet_name='Sheet1', index=False)

# Get the xlsxwriter objects from the dataframe writer object
workbook  = ___
worksheet = ___.sheets['Sheet1']


# Set green fill with dark green text format
green_format = workbook.add_format({'bg_color': '#C6EFCE',
'font_color': '#006100'})

# Format cells of boroughs with land area over 50 sq miles
# with green format
worksheet.___('B2:B6', {'type': 'cell',
'criteria': 'greater than',
'value': ___,
'format': ___})

# Format two-grade color scale to the population cells
worksheet.conditional_format('C2:C6', {___: '2_color_scale'})

# Close the workbook
___